## Dilemmas:
1. Maybe paragraph corpus, and not text (because the B1 articles have more than one, while B2-C1 articles have only one)
2. With regular news articles?

In [29]:
%load_ext autoreload
%autoreload 2

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


In [30]:
import numpy as np
import pandas as pd
import seaborn as sns
import matplotlib.pylab as plt
from IPython.display import IFrame

import fetch_dw

plt.ion()

In [ ]:
pages_df = fetch_dw.build_pages_df(1000 , to_filter=False)

  0%|          | 0/3 [00:00<?, ?it/s]

Retrieving all articles URLS...


  0%|          | 0/201 [00:00<?, ?it/s]

Retrieving all the pages...


  2%|▏         | 4/201 [00:04<03:22,  1.03s/it]

## URL type (`av`)

In [ ]:
url_types = pages_df["url"].apply(lambda s: s.split("/")[-1].split("-")[0])

In [ ]:
url_types.value_counts()

In [ ]:
pages_df[url_types == "av"]["artikel"].value_counts()

In [ ]:
pages_df = pages_df[url_types != "av"]

## Levels vs. Type of Article (`artikel`)

In [ ]:
leveled_artikel_count = pages_df.groupby(["artikel", "levels"]).size()
leveled_artikel_count[leveled_artikel_count>5].dropna(how="all").fillna(0).sort_values()

### TAKEN TO CORPUS

#### Top-Thema – Podcast - B1 [One Text]


In [ ]:
page = pages_df[pages_df["artikel"] == "Top-Thema – Podcast"].iloc[0]
print(page["url"])
#print(page["content"])
IFrame("http://dw.com" 
       + page["url"],
       width=1000, height=500)

#### Nachrichten - B2 & C1 - [Multiple Texts]

In [ ]:
page = pages_df[pages_df["artikel"] == "Nachrichten"].iloc[0]
#print(page["content"])
IFrame("http://dw.com" 
       + page["url"],
       width=1000, height=500)

#### Langsam gesprochene Nachrichten - B2 & C1 [Multiple Texts]

In [ ]:
page = pages_df[pages_df["artikel"] == "Langsam gesprochene Nachrichten"].iloc[0]
#print(page["content"])
IFrame("http://dw.com" 
       + page["url"],
       width=1000, height=500)

#### Top-Thema – Lektionen - B1 [One Text]

In [ ]:
page = pages_df[pages_df["artikel"] == "Top-Thema – Lektionen"].iloc[0]
#print(page["content"])
IFrame("http://dw.com" 
       + page["url"],
       width=1000, height=500)

### NOT TAKEN TO CORPUS

#### Video-Thema – Lektionen - B2 & C1 [Dialogue]

In [ ]:
page = pages_df[pages_df["artikel"] == "Video-Thema – Lektionen"].iloc[0]
#print(page["content"])
IFrame("http://dw.com" 
       + page["url"],
       width=1000, height=500)

#### Sprachbar - C1 & C2

In [ ]:
page = pages_df[pages_df["artikel"] == "Sprachbar"].iloc[0]
#print(page["content"])
IFrame("http://dw.com" 
       + page["url"],
       width=1000, height=500)

#### Alltagsdeutsch – Podcast - C1 & C2 [No Text at all]

In [ ]:
page = pages_df[pages_df["artikel"] == "Alltagsdeutsch"].iloc[0]
#print(page["content"])
IFrame("http://dw.com" 
       + page["url"],
       width=1000, height=500)

#### Video-Thema – Podcast ohne UT - B2 & C1

In [ ]:
page = pages_df[pages_df["artikel"] == "Video-Thema – Podcast ohne UT"].iloc[0]
#print(page["content"])
IFrame("http://dw.com" 
       + page["url"],
       width=1000, height=500)

#### Wort der Woche  - B2, B2 & C1

In [ ]:
page = pages_df[pages_df["artikel"] == "Wort der Woche"].iloc[0]
#print(page["content"])
IFrame("http://dw.com" 
       + page["url"],
       width=1000, height=500)

## Load Full Text Dateframe

In [ ]:
pages_df = pd.read_hdf('dw.h5', "pages_df")
text_df = pd.read_hdf('dw.h5', "text_df")
paragraphs_df = pd.read_hdf('dw.h5', "paragraphs_df")

## Text Corpus QA

In [ ]:
sns.countplot(text_df["y"])

In [ ]:
text_df["wc"] = text_df["text"].apply(lambda x: len(x.split()))
text_df["pc"] = text_df["text"].apply(lambda x: len(x.split("\n")))

In [ ]:
sns.distplot(text_df[text_df["y"] == 0]["wc"], label="0")
sns.distplot(text_df[text_df["y"] == 1]["wc"], label="1")
plt.legend()

In [ ]:
text_df[text_df["y"] == 1]["pc"].describe(())

In [ ]:
text_df[text_df["y"] == 0]["pc"].value_counts()

In [ ]:
sns.violinplot(x="y", y ="wc", data=text_df)

In [ ]:
sns.violinplot(x="artikel", y ="wc", data=text_df)

In [ ]:
(text_df["text"]
 .apply(lambda text: max(zip(map(len, text.split()), text.split())))).sort_values().tail()

In [ ]:
from textstat.textstat import textstat
text_df["FKG"] = text_df["text"].apply(textstat.flesch_kincaid_grade)

sns.distplot(text_df[text_df["y"] == 0]["FKG"], label="0")
sns.distplot(text_df[text_df["y"] == 1]["FKG"], label="1")

plt.legend()
plt.title("Distribution Flesch-Kincaid Grade per Text by Reading Lables")
plt.savefig("dist-fkg.png", dpi=300)

## Parapgraphs Corpus Analysis

### BUG: LEKTION articles does not split well to paragraphs

In [ ]:
paragraphs_df = paragraphs_df[paragraphs_df["rubrik"] != "LEKTIONEN"]

In [ ]:
sns.countplot(paragraphs_df["y"])

In [ ]:
paragraphs_df["wc"] = paragraphs_df["text"].apply(lambda x: len(x.split()))
paragraphs_df["pc"] = paragraphs_df["text"].apply(lambda x: len(x.split("\n")))

In [ ]:
sns.distplot(paragraphs_df[paragraphs_df["y"] == 0]["wc"], label="0")
sns.distplot(paragraphs_df[paragraphs_df["y"] == 1]["wc"], label="1")
plt.legend()

In [ ]:
paragraphs_df["pc"].value_counts()

In [ ]:
sns.violinplot(x="y", y ="wc", data=paragraphs_df)

In [ ]:
sns.violinplot(x="artikel", y ="wc", data=paragraphs_df)

In [ ]:
from textstat.textstat import textstat
paragraphs_df["FKG"] = paragraphs_df["text"].apply(textstat.flesch_kincaid_grade)

sns.distplot(paragraphs_df[paragraphs_df["y"] == 0]["FKG"], label="0")
sns.distplot(paragraphs_df[paragraphs_df["y"] == 1]["FKG"], label="1")

plt.legend()
plt.title("Distribution Flesch-Kincaid Grade per Text by Reading Lables")
plt.savefig("dist-fkg.png", dpi=300)

## Corpus Labels Anlyasis 

## Text

### TF-IDF

In [ ]:
from sklearn.feature_extraction.text import TfidfVectorizer, CountVectorizer

no_features = 1000

tf_vectorizer = CountVectorizer(max_df=0.95, min_df=2, max_features=no_features, stop_words='english')
tf_vectors = tf_vectorizer.fit_transform(text_df["text"])
tf_feature_names = tf_vectorizer.get_feature_names()

### t-SNF by Articles & `y`

In [ ]:
from sklearn.manifold import TSNE
from sklearn.decomposition import TruncatedSVD

X_reduced = TruncatedSVD(n_components=50, random_state=0).fit_transform(tf_vectors)
X_embedded = TSNE(n_components=2, perplexity=40, verbose=2).fit_transform(X_reduced)

In [ ]:
fig = plt.figure(figsize=(10, 10))
ax = plt.axes(frameon=False)
plt.setp(ax, xticks=(), yticks=())
plt.subplots_adjust(left=0.0, bottom=0.0, right=1.0, top=0.9,
                wspace=0.0, hspace=0.0)
plt.scatter(X_embedded[:, 0], X_embedded[:, 1],
        c=text_df["y"], marker="x")

### PCA

In [ ]:
from sklearn.decomposition import PCA

X_embedded = PCA(n_components=2, random_state=0).fit_transform(tf_vectors.toarray())

In [ ]:
fig = plt.figure(figsize=(10, 10))
ax = plt.axes(frameon=False)
plt.setp(ax, xticks=(), yticks=())
plt.subplots_adjust(left=0.0, bottom=0.0, right=1.0, top=0.9,
                wspace=0.0, hspace=0.0)
plt.scatter(X_embedded[:, 0], X_embedded[:, 1],
        c=text_df["y"], marker="x")

### SVM

In [ ]:
from sklearn.linear_model import SGDClassifier

text_clf_svm = SGDClassifier(loss='hinge', penalty='l2', alpha=1e-3, n_iter=5, random_state=0)
text_clf_svm.fit(tf_vectors.toarray(), text_df["y"].as_matrix())
predicted_svm = text_clf_svm.predict(tf_vectors.toarray())
np.mean(predicted_svm == text_df["y"])


## Paragraphs

In [ ]:
from sklearn.feature_extraction.text import TfidfVectorizer, CountVectorizer

no_features = 1000

tf_vectorizer = CountVectorizer(max_df=0.95, min_df=2, max_features=no_features, stop_words='english')
tf_vectors = tf_vectorizer.fit_transform(paragraphs_df["text"])
tf_feature_names = tf_vectorizer.get_feature_names()

In [ ]:
from sklearn.manifold import TSNE
from sklearn.decomposition import TruncatedSVD


X_reduced = TruncatedSVD(n_components=50, random_state=0).fit_transform(tf_vectors)
X_embedded = TSNE(n_components=2, perplexity=40, verbose=2).fit_transform(X_reduced)

In [ ]:
fig = plt.figure(figsize=(10, 10))
ax = plt.axes(frameon=False)
plt.setp(ax, xticks=(), yticks=())
plt.subplots_adjust(left=0.0, bottom=0.0, right=1.0, top=0.9,
                wspace=0.0, hspace=0.0)
plt.scatter(X_embedded[:, 0], X_embedded[:, 1],
        c=paragraphs_df["y"], marker="x")

### PCA

In [ ]:
X_embedded = PCA(n_components=2, random_state=0).fit_transform(tf_vectors.toarray())

In [ ]:
fig = plt.figure(figsize=(10, 10))
ax = plt.axes(frameon=False)
plt.setp(ax, xticks=(), yticks=())
plt.subplots_adjust(left=0.0, bottom=0.0, right=1.0, top=0.9,
                wspace=0.0, hspace=0.0)
plt.scatter(X_embedded[:, 0], X_embedded[:, 1],
        c=paragraphs_df["y"], marker="x")

In [ ]:
from sklearn.linear_model import SGDClassifier

text_clf_svm = SGDClassifier(loss='hinge', penalty='l2', alpha=1e-3, n_iter=5, random_state=0)
text_clf_svm.fit(tf_vectors.toarray(), paragraphs_df["y"].as_matrix())
predicted_svm = text_clf_svm.predict(tf_vectors.toarray())
np.mean(predicted_svm == paragraphs_df["y"])


## LDA

In [ ]:
from sklearn.decomposition import LatentDirichletAllocation

no_features = 1000
no_topics = 20
no_top_words = 10

# LDA can only use raw term counts for LDA because it is a probabilistic graphical model
print("Preparing Data...")
tf_vectorizer = CountVectorizer(max_df=0.95, min_df=2, max_features=no_features, stop_words='english')
tf = tf_vectorizer.fit_transform(text_df["text"])
tf_feature_names = tf_vectorizer.get_feature_names()

# Run LDA
print("Running LDA...")
lda = LatentDirichletAllocation(n_topics=no_topics, max_iter=5, learning_method='online', learning_offset=50.,random_state=0, verbose=True).fit(tf)
print("Done!")

def display_topics(model, feature_names, no_top_words):
    for topic_idx, topic in enumerate(model.components_):
        print("Topic %d:" % (topic_idx))
        print(" ".join([feature_names[i]
                        for i in topic.argsort()[:-no_top_words - 1:-1]]))

display_topics(lda, tf_feature_names, no_top_words)